#### This file creates runtime.csv, which records the runtime for each solution in seconds (on each testcase)

In [ ]:
import os
import numpy as np
import pandas as pd 
from statistics import mean 

In [ ]:
home_dir: str = os.getcwd()
logs_dir: str = os.path.join(home_dir, 'logs')
plot_dir: str = os.path.join(home_dir, 'plots')
sub_dir: str = os.path.join(home_dir, 'subA3')
info_task1_dir: str = os.path.join(home_dir, 'info_task1')
info_task2_dir: str = os.path.join(home_dir, 'info_task2') 
results_dir: str = os.path.join(home_dir, 'results')

print (home_dir, logs_dir, plot_dir, sub_dir, results_dir)

In [ ]:
categories = ['medium'] # decides for which category marks need to calulate

weightage

In [ ]:
weights = {'medium': 100}
task1_categories = {'medium': [10]}
task2_categories = {'medium': [10, 11, 12]}
total_test_cases = {'medium': 0}
for category in categories: # 'large']:
    if category in task1_categories:
        total_test_cases[category] = len(task1_categories[category]) + len(task2_categories[category])
marksFor = dict()

kerberos from Entry number

In [ ]:
def get_kerberos(Entry_no: str) -> str:
    year, dept, endwith = Entry_no[2: 4], Entry_no[4: 7], Entry_no[7: ]
    kerberos = dept + year + endwith
    return kerberos.lower()

Get the mapping of students

In [ ]:
all_students = os.listdir(os.path.join(home_dir, 'subA3'))
all_students = [student.split('_')[1] for student in all_students]

In [ ]:
#  For each test case within category hold the equal weight
marksFor = dict()
for category, testcases in task1_categories.items():
    uniform_weight = weights[category] / total_test_cases[category]
    for testid in testcases:
        marksFor[f'task1_test{testid}'] = uniform_weight
for category, testcases in task2_categories.items():
    uniform_weight = weights[category] / total_test_cases[category]
    for testid in testcases:
        marksFor[f'task2_test{testid}'] = uniform_weight

marksFor

For task 1 : Runtime

In [ ]:
# 1. Fetch the startk and endk for each test case 
range_of_k = dict()

for testcases in task1_categories.values():
    for testid in testcases:
        info_file = (os.path.join(info_task1_dir, f'test{testid}', 'task1_info.txt'))
        if os.path.exists(info_file):
            startk, endk = None, None
            with open(info_file, 'r') as f:
                line: str = f.readline().split(',')
                startk: int = int((line[0]).split('=')[1])
                endk: int = int((line[1]).split('=')[1])
                f.close()
            range_of_k[testid] = (startk, endk)
            
# 2. Get the columns for the dataframes
alltestcases = []
for testcases in task1_categories.values():
    for testid in testcases:
        alltestcases.append(f'task1_test{testid}')
for testcases in task2_categories.values():
    for testid in testcases:
        alltestcases.append(f'task2_test{testid}')

# 3. Make the dataframe
student_df = pd.DataFrame(columns= ['Entry_no'] + alltestcases + ['Total Score'])
student_df['Entry_no'] = all_students
student_df.set_index('Entry_no', inplace=True)
student_df = student_df.replace(np.nan, 0)

# 4. Filter the runtime for each student
student_marks = {testlabel:dict() for testlabel in alltestcases}

for testcases in task1_categories.values():
    for testid in testcases:
        test_logs_dir = os.path.join(logs_dir, f'task1_logs_{testid}')
        if os.path.exists(test_logs_dir):
            verdict_file = os.path.join(test_logs_dir, f'time_test_{testid}.csv')
            df = pd.read_csv(verdict_file)
            df.rename(columns={'Total Run Time (if correct)': 'Runtime(in sec)'}, inplace=True)
            print (df.columns)
            startk, endk = range_of_k[testid]
            TIMECOL = 'Runtime(in sec)'
            for _, student in df.iterrows():
                entry_no = student['Entry_number']
                _entry_no = entry_no.replace('.txt', '')
                run_time = float(student[TIMECOL])
                student_marks[f'task1_test{testid}'][_entry_no] = round(run_time, 2)

For task 2 

In [ ]:
# 1. Fetch the k and p for each test case 
k, p = -1, -1
meta_data = dict()
for testcases in task2_categories.values():
    for testid in testcases:
        info_file = (os.path.join(info_task2_dir, f'test{testid}', 'task2_info.txt'))
        if os.path.exists(info_file):
            k, p = None, None
            with open(info_file, 'r') as f:
                line: str = f.readline().split(',')
                k: int = int((line[0]).split('=')[1])
                p: int = int((line[1]).split('=')[1])
                f.close()
            meta_data[testid] = (k, p)

# 2. There are single k, p only. Filter the results of each student
for testcases in task2_categories.values():
    for testid in testcases:
        test_logs_dir = os.path.join(logs_dir, f'task2_logs_{testid}')
        if os.path.exists(test_logs_dir):
            verdict_file = os.path.join(test_logs_dir, f'time_test_{testid}.csv')
            df = pd.read_csv(verdict_file)
            df = df.replace('Invalid', np.nan)
            k, p = meta_data[testid]
            # Since, there is just a single column in results
            verdict = f'Verdict: GroupSize={k}, and p = {p}'
            TIMECOL = 'Runtime(in sec)'
            for _, student in df.iterrows():
                entry_no = student['Entry_number']
                _entry_no = entry_no.replace('.txt', '')
                run_time = float(student[TIMECOL])
                print (f'testid = {testid}, entry_no = {_entry_no} and run_time = {run_time}')
                student_marks[f'task2_test{testid}'][_entry_no] = run_time 


for testid, scores in student_marks.items():
    for eno, s in scores.items():
        student_df.loc[eno, testid] = s
student_df = student_df.replace(0, np.nan)
student_df['kerberos'] = student_df.index.map(get_kerberos)
student_df.head()
student_df.to_csv(os.path.join(results_dir, 'runtime.csv'))


In [ ]:

def get_marks(store_runtimes, x):
    n = len(store_runtimes)
    top10 = (n * 10) // 100 
    top70 = (n * 70) // 100
    idx = store_runtimes.index(x)
    if idx <= top10:
        x1 = store_runtimes[0]
        idx10 = (n * 10) // 100
        x2 = store_runtimes[idx10] + 20

        y1 = 25
        y2 = 25
        m = (y2 - y1) / (x2 - x1)
        '''
        24 = m(x1) + c
        c = 24 - m(x1)
        '''
        c = y1 - m * x1 
        return m * x + c

    elif idx > top10 and idx <= top70:
        idx10 = (n * 10) // 100
        x1 = store_runtimes[idx10]
        
        idx70 = (n * 70) // 100
        x2 = store_runtimes[idx70] + 100

        y1 = 24.5
        y2 = 19.5
        m = (y2 - y1) / (x2 - x1)
        '''
        24 = m(x1) + c
        c = 24 - m(x1)
        '''
        c = y1 - m * x1 
        return m * x + c
    elif idx > top70:
        idx70 = (n * 70) // 100
        x1 = store_runtimes[idx70 + 1]
        x2 = store_runtimes[-1] + 200
        y1 = 19.5
        y2 = 11
        m = (y2 - y1) / (x2 - x1) 
        c = y1 - m * x1 
        return m * x + c 


final_df = student_df.copy()
# For each test cases check the run-time of each student.
for category in categories:
    for testid in task1_categories[category]:
        testid = runtime = f'task1_test{testid}'
        store_runtimes = final_df[runtime]
        store_runtimes = list(filter(lambda time: (time != np.nan and time > 0.0), store_runtimes))
        store_runtimes.sort()
        n = len(store_runtimes)

        # fastest = mean(store_runtimes[:5])
        # mid = 6 * fastest 

        for idx, student in final_df.iterrows():
            if (student[runtime] is not np.nan) and (student[runtime] > 0.0):
                t = student[runtime]
                final_df.at[idx, runtime] = get_marks(store_runtimes, t)
                # score = max(40, 100 - (max(0, (t - fastest)) / (mid - fastest)) * 25)
                # final_df.at[idx, runtime] = (score * marksFor[testid]) / 100
            else:
                final_df.at[idx, runtime] = 0.0
            
    for testid in task2_categories[category]:
        testid = runtime = f'task2_test{testid}'
        store_runtimes = final_df[runtime]
        store_runtimes = list(filter(lambda time: (time != np.nan and time > 0.0), store_runtimes))
        store_runtimes.sort()
        fastest = mean(store_runtimes[:5])
        mid = 6 * fastest 
        for idx, student in final_df.iterrows():
            if (student[runtime] is not np.nan) and (student[runtime] > 0.0):
                t = student[runtime]
                final_df.at[idx, runtime] = get_marks(store_runtimes, t)
                # score = max(40, 100 - (max(0, (t - fastest)) / (mid - fastest)) * 25)
                # final_df.at[idx, runtime] = (score * marksFor[testid]) / 100 
            else:
                final_df.at[idx, runtime] = 0.0

cols = list(final_df.columns)
cols.remove('Total Score')
cols.remove('kerberos')
final_df['Total Score'] = final_df.loc[:, cols].sum(axis='columns')
print (final_df.head())
# final_df.set_index('kerberos', inplace=True)
final_df.to_csv(os.path.join(results_dir, 'runtime_score.csv')) # Store the marks assigned to each runtime

In [ ]:

final_df = pd.read_csv(os.path.join(results_dir, 'runtime_score.csv'))
runtime_df = pd.read_csv(os.path.join(results_dir, 'runtime.csv'))

import matplotlib.pyplot as plt
import numpy as np


for category in categories:
    for testid in task1_categories[category]:
        testid = runtime = f'task1_test{testid}'
        scores = final_df[testid].tolist()
        runtime = runtime_df[testid].tolist()

        scores = [s for s in scores if s > 0.0]
        runtime = [int(r) for r in runtime if ((r is not np.nan) and (r > 0.0))]
        scores = np.array(scores)
        runtime = np.array(runtime)
        pairs = [(r, s) for r, s in zip(runtime, scores)]
        pairs.sort()
        scores = [s for r, s in pairs]
        runtime = [r for r, _ in pairs]
        print (pairs)
        plt.xlabel('Runtime (in seconds)')
        plt.ylabel('Score')
        plt.title(f'Test case {testid}')
        plt.xticks(range(0, max(runtime), 50), rotation=45)
        
        plt.plot(runtime, scores, marker='o', markersize='5', mfc='red')
        plt.gcf().set_size_inches(13, 6)
        plt.savefig(os.path.join(plot_dir, f'task1_test{testid}.png'), format='png')
        plt.show()

for testcases in task2_categories.values():
    for testid in task2_categories[category]:
        testid = runtime = f'task2_test{testid}'
        scores = final_df[testid].tolist()
        runtime = runtime_df[testid].tolist()

        scores = [s for s in scores if s > 0.0]
        runtime = [int(r) for r in runtime if ((r is not np.nan) and (r > 0.0))]
        scores = np.array(scores)
        runtime = np.array(runtime)
        pairs = [(r, s) for r, s in zip(runtime, scores)]
        pairs.sort()
        scores = [s for r, s in pairs]
        runtime = [r for r, _ in pairs]
        print (pairs)
        
        plt.xlabel('Runtime (in seconds)')
        plt.ylabel('Score')
        plt.xticks(range(0, max(runtime), 50), rotation=45)

        plt.title(f'Test case {testid}')

        plt.plot(runtime, scores, marker='o', markersize='5', mfc='red')
        plt.gcf().set_size_inches(13, 6)
        plt.savefig(os.path.join(plot_dir, f'task2_test{testid}.png'), format='png')
        plt.show()

In [ ]:
# import matplotlib.pyplot as plt
# from matplotlib.ticker import AutoMinorLocator
# from matplotlib import gridspec
# import pandas as pd 
# import os 

# runtime_df = pd.read_csv(os.path.join(home_dir, 'runtime.csv'))
# runtime_df.set_index('Entry_no', inplace=True)
# runtime_df.drop(columns=['Total Score'], inplace=True)
# for testid in runtime_df.columns:
#     with open(os.path.join(home_dir, f'test{testid}', 'task1_info.txt')) as f: 
#         lines = f.readlines()
#         print (lines)

#     times = list(runtime_df[testid])
#     times = list(filter(lambda time: (time != np.nan and time > 0), times))
#     times.sort()

#     bins = np.arange(min(times) - 20, max(times) + 30, 15)
#     plt.xlim([min(times)- 10, max(times) + 10])
#     plt.hist(times, bins=bins, histtype='bar', rwidth=0.9, alpha=0.5)
#     plt.xlabel('Run time (seconds)')
#     plt.ylabel('Number of students')
#     plt.title(f'Test case {testid}')
#     if int(testid) >= 5: 
#         plt.savefig(os.path.join(plot_dir, f'test{testid}.png'), format='png')
#     plt.show()
#     print (f'testid = {testid}, {times}')
